# Dual numbers

## definition
Like imaginary numbers, a dual number is pair of real numbers with the defined constant $\epsilon$ such that $\epsilon^2 = 0$. So, $\forall a,b \in \mathbb{R}$,
$$(a+b\epsilon)(a-b\epsilon) = a^2$$
and
$$(a+b\epsilon)^n = a^n + na^{n-1}b\epsilon$$
where the second property can be easily proved by induction,
$$
\begin{aligned}
(a+b\epsilon)^{n+1}
&=
(a+b\epsilon) (a+b\epsilon)^n
\\&=
(a+b\epsilon) \left(a^n + na^{n-1}b\epsilon\right)
\\&=
a^{n+1} + na^n b\epsilon + a^nb\epsilon
\\&=
a^{n+1} + (n+1)a^n b\epsilon .
\end{aligned}
$$

Considering $P(x) = \sum_n a_n x^n$, then
$$
\begin{aligned}
P(x+h\epsilon) 
&=
\sum_n a_n (x+h\epsilon)^n
\\&=
\sum_n a_n x^n + \sum_n n a_n x^{n-1} h\epsilon
\\&=
P(x) + P'(x)h\epsilon .
\end{aligned}
$$
So, for any analytical function $f$,
$$f(x+h\epsilon) = f(x) + f'(x)h\epsilon.$$

Let $f$ and $g$ be two analytical functions, the arithmetic operations plus the function composition of $f$ and $g$ applied to a dual number correspond to all the derivative rules including the chain rule,
$$
\begin{aligned}
f(x+h\epsilon) \pm g(x+h\epsilon) 
&= f(x) + f'(x)h\epsilon \pm g(x) \pm g'(x)h\epsilon
\\&= \left( f(x) \pm g(x) \right) + \left( f'(x) \pm g'(x) \right)h\epsilon ,
\\[1em]
f(x+h\epsilon) \times g(x+h\epsilon) 
&= \left( f(x) + f'(x)h\epsilon \right) \times \left( g(x) \pm g'(x)h\epsilon \right)
\\&= f(x)g(x) + \left( f'(x)g(x) + f(x)g'(x) \right)h\epsilon ,
\\[1em]
\frac{f(x+h\epsilon)}{g(x+h\epsilon)}
&= \frac{f(x+h\epsilon)}{g(x+h\epsilon)} \times \frac{g(x-h\epsilon)}{g(x-h\epsilon)}
\\&= \frac{f(x)g(x) + \left( f'(x)g(x) - f(x)g'(x) \right)h\epsilon}{g(x)^2}
\\&= \frac{f(x)}{g(x)} + \frac{f'(x)g(x) - f(x)g'(x)}{g(x)^2}h\epsilon ,
\\[1em]
f(g(x+h\epsilon))
&= f(g(x) + g'(x)h\epsilon)
\\&= f(g(x)) + f'(g(x)) g'(x) h\epsilon .
\end{aligned}
$$

## implementing dual numbers in julia
We first start with a structure to associate $f$ and $f'$.

In [1]:
struct Dual <: Number
    x::Real
    dx::Real
end

Next, we define how to convert a variable $x$ into a dual number.

In [2]:
Dual(x) = Dual(x, one(x));

Then we define how to convert constant numbers into dual numbers.

In [3]:
Base.convert(::Type{Dual}, x::Real) = Dual(x, zero(x))

And how to typecast any real number into a dual number.

In [4]:
Base.promote_rule(::Type{Dual}, ::Type{<:Real}) = Dual

The next step is define the arithmetic properties.

In [5]:
Base.:+(f::Dual, g::Dual) = Dual(f.x + g.x, f.dx + g.dx)
Base.:-(f::Dual, g::Dual) = Dual(f.x - g.x, f.dx - g.dx)
Base.:*(f::Dual, g::Dual) = Dual(f.x * g.x, f.dx * g.x + f.x * g.dx)
Base.:/(f::Dual, g::Dual) = Dual(f.x / g.x, (f.dx * g.x - f.x * g.dx) / g.x^2)
# also the negative of a dual number
Base.:-(f::Dual) = Dual(-f.val, -f.dx)

To make things easy we can define the constant $\epsilon$.

In [6]:
const ϵ = Dual(0,1)
ϵ^2

Dual(0, 0)

We can make it more visual by define a print format.

In [7]:
Base.show(io::IO, f::Dual) = print(io, "$(f.x) + $(f.dx)ϵ")
ϵ^2

0 + 0ϵ

Now, let's define how to use some standard functions with dual numbers. Thanks to the multiple dispatch we can add more functions to this list later.

In [8]:
Base.sin(f::Dual) = sin(f.x) + cos(f.x) * f.dx * ϵ
Base.cos(f::Dual) = cos(f.x) - sin(f.x) * f.dx * ϵ
Base.exp(f::Dual) = exp(f.x) + exp(f.x) * f.dx * ϵ
Base.log(f::Dual) = log(f.x) + f.dx / f.x * ϵ
Base.sqrt(f::Dual) = sqrt(f.x) + f.dx / sqrt(f.x) * ϵ

In [9]:
f(x) = x^2 + 2x - cos(sqrt(x))
f(Dual(π))

16.353083249392316 + 8.83594211490989ϵ

## Automatic differentiation for many variables
The next natural step is define how functions with two or more variables handle with dual numbers.

Starting by taking $f:\mathbb{R}^2 \to \mathbb{R}$,
$$
f(x + h_x \epsilon_1, y + h_y \epsilon_2) = f(x,y) +\partial_x f(x,y) h_x \epsilon_1 + \partial_y f(x,y) h_y \epsilon_2
$$
where $\epsilon_1 \epsilon_2 = \epsilon_1^2 = \epsilon_2^2 = 0$, we could say the variables are
$$
\begin{aligned}
x + h_x \epsilon_1 &= x + \begin{pmatrix}h_x & 0\end{pmatrix}
\\
y + h_y \epsilon_2 &= y + \begin{pmatrix}0 & h_y\end{pmatrix}
\end{aligned}
$$
so we could rewrite the previous expression as
$$
\begin{aligned}
f(x + h_x \epsilon_1, y + h_y \epsilon_2) &= 
f(x,y) + \begin{pmatrix}\partial_xf(x,y)h_x & \partial_yf(x,y)h_y\end{pmatrix} \\&=
f(x,y) + \nabla f(x,y) .
\end{aligned}
$$

## implementing a multivariate dual number in julia
We start by define

In [10]:
struct DualN <: Number
    val::Real
    grad::Vector{Real}
end

Next we define a constructor that takes $N$, $(x_1,x_2,x_3\dots)$ and a vector of $N$ dual numbers
$$
\begin{pmatrix}
x_1 + \epsilon_1\\
x_2 + \epsilon_2\\
x_3 + \epsilon_3\\
\vdots
\end{pmatrix}
$$ 

In [11]:
using LinearAlgebra: I, Matrix

function DualN(vars::Vector{<:Real})
    l = length(vars)
    id = Matrix(I, l, l)
    map(1:l) do j
        DualN(vars[j], id[:,j])
    end
end;

Once again we need to be able to convert constants into dual numbers.

In [12]:
Base.convert(::Type{DualN}, x::Real) = DualN(x, [zero(x)])
Base.promote_rule(::Type{DualN}, ::Type{<:Real}) = DualN

And then we define the arithmetic operations.

In [13]:
Base.:+(f::DualN, g::DualN) = DualN(f.val + g.val, f.grad .+ g.grad)
Base.:-(f::DualN, g::DualN) = DualN(f.val - g.val, f.grad .- g.grad)
Base.:*(f::DualN, g::DualN) = DualN(f.val * g.val, f.grad .* g.val .+ f.val * g.grad)
Base.:/(f::DualN, g::DualN) = DualN(f.val / g.val, (f.grad .* g.val .- f.val .* g.grad) ./ g.val^2)
# also the negative of a dual number
Base.:-(f::DualN) = DualN(-f.val, -f.grad)

First let's try a $\mathbb{R}^2 \to \mathbb{R}$.

In [14]:
f(x,y) = x^2 - y^2

f(DualN([-1,1])...)

DualN(0, Real[-2, -2])

Next let's try a function that takes a vector as input.

In [15]:
entropy(λ) = -λ'log.(λ)

entropy (generic function with 1 method)

In order to be able to use the funcion `entropy` with dual numbers we need to extend the functions `log` and `conj` since in julia the operation `A'A` is equiavlent to $A^\dagger A$.

In [16]:
Base.log(f::DualN) = DualN(log(f.val), f.grad ./ f.val)
Base.conj(f::DualN) = f

In [17]:
entropy(DualN([2/3, 2/9, 1/9]))

DualN(0.8486855577264172, Real[-0.5945348918918356, 0.5040773967762742, 1.1972245773362196])

## propagation of uncertainty using dual numbers
Assuming we have independence between variables, the error $\sigma_f$ of a function $f(x_1,x_2,\dots)$ can be obtained by calculate
$$
\sigma_f = \sqrt{\Delta^T \Delta}
$$
where $\Delta = \nabla f \odot \vec{\sigma}$ and $\odot$ indicates the elementwise product.

In [18]:
uncert(f::DualN, σ::Vector{<:Real}) = let
    Δ = f.grad .* σ
    (val = f.val, err = √(Δ'Δ))
end

uncert (generic function with 1 method)

So, as a final example,

In [19]:
p = [2/3, 2/9, 1/9]
σ = [0.1, 0.12, 0.08]
s = entropy(DualN(p))
uncert(s, σ)

(val = 0.8486855577264172, err = 0.12793392864918046)